In [1]:
import numpy as np
import pandas as pd
import csv
import json
import math
import pyreadstat
# Suppressing scientific notation in pandas? - Stack Overflow
pd.set_option('display.float_format', lambda x: '%.3f' % x)

For age, 

- we are going to use the fixed year of birth (YOB) groups and convert them (survey year – YOB) into age ranges. 
- For each survey year (1995-2017), we will set the minimum age at 16, which will be indicated in the final results. 
- The imputed age (R_AGE_IMP in 2017 and R_AGE in 1995, 2001, and 2009) will be the specific age variable used in the final tables. 
- We will not include any other years of birth other than those indicated in your prior email: 1946-1964, 1965-1980, 1981-1996, 1997-2012

A crosswalk table for YOB (age group) for each survey year


| 1995                |2001                 |2009                 |2017
| ------------------- | ------------------- | ------------------- | ------------------- |
|1946-1964 (age 31-49)|1946-1964 (age 37-55)|1946-1964 (age 45-63)|1946-1964 (age 53-71)|
|1965-1979 (age 16-30)|1965-1980 (age 21-36)|1965-1980 (age 29-44)|1965-1980 (age 37-52)|
|n.a.                 |1981-1985 (age 16-20)|1981-1993 (age 16-28)|1981-1996 (age 21-36)|
|n.a.                 |n.a.                 |n.a.                 |1997-2001 (age 16-20)|

For the trip purpose, we will use variable WHYTRP90 with the following categories:
1.	Work (01–To/From Work + 02–Work-Related Business)
2.	School/church (05–School/Church)
3.	Other social/recreational (10–Other Social/Recreational)
4.	Shopping and family/personal business (03–Shopping + 04–Other Family/Personal Business)
5.	Visit friends/relatives (08– Visit Friends/Relatives)

For each survey year, you will see four tables:
1.	Annualized weighted sum for person trips (PT) by age group (YOB based) by trip purpose for all 7 days.
2.	Sample size for person trips (PT) by age group (YOB based) by trip purpose for all 7 days.
3.	Annualized weighted sum for person trips (PT) by age group (YOB based) by trip purpose for weekdays (M-F) only.
4.	Sample size for person trips (PT) by age group (YOB based) by trip purpose for weekdays (M-F) only.

Hence, in total 16 tables will be generated.


#### 2017 NHTS

In [2]:
# Import the trip public file from local file in to a dataframe
f_trippub = r'E:\GIS_Data\Layla2019_NHTS_backup\2017csv_v1.2\trippub.csv'
trip = pd.read_csv(f_trippub)

In [4]:
# Recode the trip purpose variable
def whytrp90_re (row):
    purp = row['WHYTRP90']
    if purp in [1, 2]:
        return '01 - Work'
    elif purp == 5:
        return '02 - School/church'
    elif purp == 10:
        return '03 - Other social/recreational'
    elif purp in [3, 4]:
        return '04 - Shopping and family/personal business'
    elif purp == 8:
        return '05 - Visit friends/relatives'

In [6]:
# Recode the R_AGE_IMP variable
# *** convert year of birth (YOB) to age: Survey year 2017 - YOB
# *** 1946-1964 --> 53-71
# *** 1965-1980 --> 37-52, 
# *** 1981-1996 --> 21-36, 
# *** 1997-2001 (instead of 2012 to keep minimum age at 16) --> 16-20

def r_age_imp_re (row):
    age = row['R_AGE_IMP']
    if age >= 53 and age <= 71:
        return '01 - 1946-1964 (age 53-71)'
    elif age >= 37 and age <= 52:
        return '02 - 1965-1980 (age 37-52)'
    elif age >= 21 and age <= 36:
        return '03 - 1981-1996 (age 21-36)'
    elif age >= 16 and age <= 20:
        return '04 - 1997-2001 (age 16-20)'

In [7]:
# To create the two variables based on the two functions above
trip['WHYTRP90_re'] = trip.apply(whytrp90_re, axis = 1)
trip['R_AGE_IMP_re'] = trip.apply(r_age_imp_re, axis = 1)
# Select variables needed for the tables
trip_1 = trip[['R_AGE_IMP', 'R_AGE_IMP_re', 'WHYTRP90', 'WHYTRP90_re', 'TRAVDAY', 'WTTRDFIN']]
# Quick preview
trip_1.head()

,R_AGE_IMP,R_AGE_IMP_re,WHYTRP90,WHYTRP90_re,TRAVDAY,WTTRDFIN
0,67,01 - 1946-1964 (age 53-71),5,02 - School/church,2,75441.906
1,67,01 - 1946-1964 (age 53-71),5,02 - School/church,2,75441.906
2,66,01 - 1946-1964 (age 53-71),1,01 - Work,2,71932.646
3,66,01 - 1946-1964 (age 53-71),1,01 - Work,2,71932.646
4,28,03 - 1981-1996 (age 21-36),5,02 - School/church,2,80122.687


In [41]:
# For all days in a week
## weighted sum
all7_sum = pd.crosstab([trip_1.R_AGE_IMP_re], [trip_1.WHYTRP90_re],
            trip_1.WTTRDFIN, aggfunc = 'sum',
            rownames=['YOB'],
            colnames=['Trip purpose'],
            dropna=True,
            margins=True)

## sample size
all7_sample = pd.crosstab([trip_1.R_AGE_IMP_re], [trip_1.WHYTRP90_re],
            trip_1.WHYTRP90_re, aggfunc = 'count',
            rownames=['YOB'],
            colnames=['Trip purpose'],
            dropna=True,
            margins=True)

# For weekdays only (M-F)
# Weekend in TRAVDAY: 1 = Sunday and 7 = Saturday
# Weekdays in TRAVDAY: 2-6 as M-F

weekdays = [2, 3, 4, 5, 6]
trip_1_wkdays = trip_1[trip_1['TRAVDAY'].isin(weekdays)]

## weighted sum
weekdays_sum = pd.crosstab([trip_1_wkdays.R_AGE_IMP_re], [trip_1_wkdays.WHYTRP90_re],
            trip_1_wkdays.WTTRDFIN, aggfunc = 'sum',
            rownames=['YOB'],
            colnames=['Trip purpose'],
            dropna=True,
            margins=True)

## sample size
weekdays_sample = pd.crosstab([trip_1_wkdays.R_AGE_IMP_re], [trip_1_wkdays.WHYTRP90_re],
            trip_1_wkdays.WHYTRP90_re, aggfunc = 'count',
            rownames=['YOB'],
            colnames=['Trip purpose'],
            dropna=True,
            margins=True)

# export all tables into one excelbook
savetofile = r'E:\GIS_Data\Layla2019_NHTS_backup\NHTS_Layla\03_UserSupport\M03_2021\Q03008_RBlair\NHTS2017_tbs.xlsx'
writer = pd.ExcelWriter(savetofile, engine='xlsxwriter')

all7_sum.to_excel(writer, sheet_name='All7_sum')
all7_sample.to_excel(writer, sheet_name='All7_sample')
weekdays_sum.to_excel(writer, sheet_name='WD5_sum')
weekdays_sample.to_excel(writer, sheet_name='WD5_sample')

writer.save()

# further data format adjustment and tabs combining are done manually within the output excel.

#### 2009 NHTS

In [13]:
# Import the trip public file from local file in to a dataframe
f_trippub2009 = r'E:\GIS_Data\Layla2019_NHTS_backup\Csv2009\DAYV2PUB.CSV'
trip2009 = pd.read_csv(f_trippub2009)

In [25]:
# Recode the R_AGE variable (2009 R_AGE is imputed)
# *** convert year of birth (YOB) to age: Survey year 2009 - YOB
# *** 1946-1964 --> 45-63,
# *** 1965-1980 --> 29-44, 
# *** 1981-1993 (instead of 1996 to keep minimum age at 16) --> 16-28, 
# *** 1997-2012 --> not included, as all below 16

def r_age_re2009 (row):
    age = row['R_AGE']
    if age >= 45 and age <= 63:
        return '01 - 1946-1964 (age 45-63)'
    elif age >= 29 and age <= 44:
        return '02 - 1965-1980 (age 29-44)'
    elif age >= 16 and age <= 28:
        return '03 - 1981-1993 (age 16-28)'

In [26]:
# To create the two variables based on the WHYTRP90_re (unchanged from 2017) and r_age_re2009 (new one above)
trip2009['WHYTRP90_re'] = trip2009.apply(whytrp90_re, axis = 1) 
trip2009['R_AGE_re'] = trip2009.apply(r_age_re2009, axis = 1)
# Select variables needed for the tables
trip2009_1 = trip2009[['R_AGE', 'R_AGE_re', 'WHYTRP90', 'WHYTRP90_re', 'TRAVDAY', 'WTTRDFIN']]
# Quick preview
trip2009_1.head()

,R_AGE,R_AGE_re,WHYTRP90,WHYTRP90_re,TRAVDAY,WTTRDFIN
0,75,None,6,None,4,68170.670
1,75,None,8,05 - Visit friends/relatives,4,68170.670
2,75,None,8,05 - Visit friends/relatives,4,68170.670
3,75,None,5,02 - School/church,4,68170.670
4,75,None,5,02 - School/church,4,68170.670


In [40]:
# For all days in a week
## weighted sum
all7_sum2009 = pd.crosstab([trip2009_1.R_AGE_re], [trip2009_1.WHYTRP90_re],
            trip2009_1.WTTRDFIN, aggfunc = 'sum',
            rownames=['YOB'],
            colnames=['Trip purpose'],
            dropna=True,
            margins=True)

## sample size
all7_sample2009 = pd.crosstab([trip2009_1.R_AGE_re], [trip2009_1.WHYTRP90_re],
            trip2009_1.WHYTRP90_re, aggfunc = 'count',
            rownames=['YOB'],
            colnames=['Trip purpose'],
            dropna=True,
            margins=True)

# For weekdays only (M-F)
# Weekend in TRAVDAY: 1 = Sunday and 7 = Saturday
# Weekdays in TRAVDAY: 2-6 as M-F

weekdays = [2, 3, 4, 5, 6]
trip2009_1_wkdays = trip2009_1[trip2009_1['TRAVDAY'].isin(weekdays)]

## weighted sum
weekdays_sum2009 = pd.crosstab([trip2009_1_wkdays.R_AGE_re], [trip2009_1_wkdays.WHYTRP90_re],
            trip2009_1_wkdays.WTTRDFIN, aggfunc = 'sum',
            rownames=['YOB'],
            colnames=['Trip purpose'],
            dropna=True,
            margins=True)

## sample size
weekdays_sample2009 = pd.crosstab([trip2009_1_wkdays.R_AGE_re], [trip2009_1_wkdays.WHYTRP90_re],
            trip2009_1_wkdays.WHYTRP90_re, aggfunc = 'count',
            rownames=['YOB'],
            colnames=['Trip purpose'],
            dropna=True,
            margins=True)

# export all tables into one excelbook
savetofile = r'E:\GIS_Data\Layla2019_NHTS_backup\NHTS_Layla\03_UserSupport\M03_2021\Q03008_RBlair\NHTS2009_tbs.xlsx'
writer = pd.ExcelWriter(savetofile, engine='xlsxwriter')

all7_sum2009.to_excel(writer, sheet_name='All7_sum2009')
all7_sample2009.to_excel(writer, sheet_name='All7_sample2009')
weekdays_sum2009.to_excel(writer, sheet_name='WD5_sum2009')
weekdays_sample2009.to_excel(writer, sheet_name='WD5_sample2009')

writer.save()

# further data format adjustment and tabs combining are done manually within the output excel.

#### 2001 NHTS

In [18]:
# Import the trip public file from local file in to a dataframe
f_trippub2001 = r'E:\GIS_Data\Layla2019_NHTS_backup\Csv2001\DAYPUB.csv'
trip2001 = pd.read_csv(f_trippub2001)

C:\Users\MSRT_GIS\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,2,125,126) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
# Recode the R_AGE variable (2001 R_AGE is imputed)
# *** convert year of birth (YOB) to age: Survey year 2001 - YOB
# *** 1946-1964 --> 37-55,
# *** 1965-1980 --> 21-36, 
# *** 1981-1985 (instead of 1996 to keep minimum age at 16) --> 16-20, 
# *** 1997-2012 --> not included, as all below 16

def r_age_re2001 (row):
    age = row['R_AGE']
    if age >= 37 and age <= 55:
        return '01 - 1946-1964 (age 37-55)'
    elif age >= 21 and age <= 36:
        return '02 - 1965-1980 (age 21-36)'
    elif age >= 16 and age <= 20:
        return '03 - 1981-1985 (age 16-20)'

In [20]:
# To create the two variables based on the WHYTRP90_re (unchanged from 2017) and r_age_re2001 (new one above)
trip2001['WHYTRP90_re'] = trip2001.apply(whytrp90_re, axis = 1) 
trip2001['R_AGE_re'] = trip2001.apply(r_age_re2001, axis = 1)
# Select variables needed for the tables
trip2001_1 = trip2001[['R_AGE', 'R_AGE_re', 'WHYTRP90', 'WHYTRP90_re', 'TRAVDAY', 'WTTRDFIN']]
# Quick preview
trip2001_1.head()

,R_AGE,R_AGE_re,WHYTRP90,WHYTRP90_re,TRAVDAY,WTTRDFIN
0,26,02 - 1965-1980 (age 21-36),10,03 - Other social/recreational,1,4448782.175
1,26,02 - 1965-1980 (age 21-36),10,03 - Other social/recreational,1,4448782.175
2,26,02 - 1965-1980 (age 21-36),10,03 - Other social/recreational,1,4448782.175
3,26,02 - 1965-1980 (age 21-36),3,04 - Shopping and family/personal business,1,4448782.175
4,26,02 - 1965-1980 (age 21-36),10,03 - Other social/recreational,1,4448782.175


In [39]:
# For all days in a week
## weighted sum
all7_sum2001 = pd.crosstab([trip2001_1.R_AGE_re], [trip2001_1.WHYTRP90_re],
            trip2001_1.WTTRDFIN, aggfunc = 'sum',
            rownames=['YOB'],
            colnames=['Trip purpose'],
            dropna=True,
            margins=True)

## sample size
all7_sample2001 = pd.crosstab([trip2001_1.R_AGE_re], [trip2001_1.WHYTRP90_re],
            trip2001_1.WHYTRP90_re, aggfunc = 'count',
            rownames=['YOB'],
            colnames=['Trip purpose'],
            dropna=True,
            margins=True)

# For weekdays only (M-F)
# Weekend in TRAVDAY: 1 = Sunday and 7 = Saturday
# Weekdays in TRAVDAY: 2-6 as M-F

weekdays = [2, 3, 4, 5, 6]
trip2001_1_wkdays = trip2001_1[trip2001_1['TRAVDAY'].isin(weekdays)]

## weighted sum
weekdays_sum2001 = pd.crosstab([trip2001_1_wkdays.R_AGE_re], [trip2001_1_wkdays.WHYTRP90_re],
            trip2001_1_wkdays.WTTRDFIN, aggfunc = 'sum',
            rownames=['YOB'],
            colnames=['Trip purpose'],
            dropna=True,
            margins=True)

## sample size
weekdays_sample2001 = pd.crosstab([trip2001_1_wkdays.R_AGE_re], [trip2001_1_wkdays.WHYTRP90_re],
            trip2001_1_wkdays.WHYTRP90_re, aggfunc = 'count',
            rownames=['YOB'],
            colnames=['Trip purpose'],
            dropna=True,
            margins=True)

# export all tables into one excelbook
savetofile = r'E:\GIS_Data\Layla2019_NHTS_backup\NHTS_Layla\03_UserSupport\M03_2021\Q03008_RBlair\NHTS2001_tbs.xlsx'
writer = pd.ExcelWriter(savetofile, engine='xlsxwriter')

all7_sum2001.to_excel(writer, sheet_name='All7_sum2001')
all7_sample2001.to_excel(writer, sheet_name='All7_sample2001')
weekdays_sum2001.to_excel(writer, sheet_name='WD5_sum2001')
weekdays_sample2001.to_excel(writer, sheet_name='WD5_sample2001')

writer.save()

# further data format adjustment and tabs combining are done manually within the output excel.

#### 1995 NPTS

In [23]:
# Import the trip public file from local file in to a dataframe
f_trippub1995 = r'E:\GIS_Data\Layla2019_NHTS_backup\SAS_transport1995\Xpt\DTRP95_2.EXP'
trip1995 = pd.read_sas(f_trippub1995, format='xport', index=None, encoding="utf-8", chunksize=None, iterator=False)

In [32]:
# Recode the trip purpose variable in 1995 as the data type is not numerical
def whytrp90_re1995 (row):
    purp = row['WHYTRP90']
    if purp in ['01', '02']:
        return '01 - Work'
    elif purp == '05':
        return '02 - School/church'
    elif purp == '10':
        return '03 - Other social/recreational'
    elif purp in ['03', '04']:
        return '04 - Shopping and family/personal business'
    elif purp == '08':
        return '05 - Visit friends/relatives'

In [42]:
# Recode the R_AGE variable (1995 R_AGE is imputed)
# *** convert year of birth (YOB) to age: Survey year 1995 - YOB
# *** 1946-1964 --> 31-49,
# *** 1965-1979 (instead of 1980 to keep minimum age at 16) --> 16-30, 
# *** 1981-1996 --> not included, as all below 16, 
# *** 1997-2012 --> not included, as all did not exist

def r_age_re1995 (row):
    age = row['R_AGE']
    if age >= 31 and age <= 49:
        return '01 - 1946-1964 (age 31-49)'
    elif age >= 16 and age <= 30:
        return '02 - 1965-1979 (age 16-30)'


In [43]:
# To create the two variables based on the WHYTRP90_re1995 and r_age_re1995
trip1995['WHYTRP90_re'] = trip1995.apply(whytrp90_re1995, axis = 1) 
trip1995['R_AGE_re'] = trip1995.apply(r_age_re1995, axis = 1)
# Select variables needed for the tables
trip1995_1 = trip1995[['R_AGE', 'R_AGE_re', 'WHYTRP90', 'WHYTRP90_re', 'TRAVDAY', 'WTTRDFIN']]
# Quick preview
trip1995_1.head()

,R_AGE,R_AGE_re,WHYTRP90,WHYTRP90_re,TRAVDAY,WTTRDFIN
0,33.000,01 - 1946-1964 (age 31-49),01,01 - Work,3.000,1553274.480
1,33.000,01 - 1946-1964 (age 31-49),01,01 - Work,3.000,1553274.480
2,52.000,None,01,01 - Work,4.000,1613507.370
3,52.000,None,01,01 - Work,4.000,1613507.370
4,26.000,02 - 1965-1979 (age 16-30),04,04 - Shopping and family/personal business,6.000,2618294.210


In [44]:
# For all days in a week
## weighted sum
all7_sum1995 = pd.crosstab([trip1995_1.R_AGE_re], [trip1995_1.WHYTRP90_re],
            trip1995_1.WTTRDFIN, aggfunc = 'sum',
            rownames=['YOB'],
            colnames=['Trip purpose'],
            dropna=True,
            margins=True)

## sample size
all7_sample1995 = pd.crosstab([trip1995_1.R_AGE_re], [trip1995_1.WHYTRP90_re],
            trip1995_1.WHYTRP90_re, aggfunc = 'count',
            rownames=['YOB'],
            colnames=['Trip purpose'],
            dropna=True,
            margins=True)

# For weekdays only (M-F)
# Weekend in TRAVDAY: 1 = Sunday and 7 = Saturday
# Weekdays in TRAVDAY: 2-6 as M-F

weekdays = [2, 3, 4, 5, 6]
trip1995_1_wkdays = trip1995_1[trip1995_1['TRAVDAY'].isin(weekdays)]

## weighted sum
weekdays_sum1995 = pd.crosstab([trip1995_1_wkdays.R_AGE_re], [trip1995_1_wkdays.WHYTRP90_re],
            trip1995_1_wkdays.WTTRDFIN, aggfunc = 'sum',
            rownames=['YOB'],
            colnames=['Trip purpose'],
            dropna=True,
            margins=True)

## sample size
weekdays_sample1995 = pd.crosstab([trip1995_1_wkdays.R_AGE_re], [trip1995_1_wkdays.WHYTRP90_re],
            trip1995_1_wkdays.WHYTRP90_re, aggfunc = 'count',
            rownames=['YOB'],
            colnames=['Trip purpose'],
            dropna=True,
            margins=True)

# export all tables into one excelbook
savetofile = r'E:\GIS_Data\Layla2019_NHTS_backup\NHTS_Layla\03_UserSupport\M03_2021\Q03008_RBlair\NHTS1995_tbs.xlsx'
writer = pd.ExcelWriter(savetofile, engine='xlsxwriter')

all7_sum1995.to_excel(writer, sheet_name='All7_sum1995')
all7_sample1995.to_excel(writer, sheet_name='All7_sample1995')
weekdays_sum1995.to_excel(writer, sheet_name='WD5_sum1995')
weekdays_sample1995.to_excel(writer, sheet_name='WD5_sample1995')

writer.save()

# further data format adjustment and tabs combining are done manually within the output excel.